In [1]:
from pyspark.sql.types import *
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 04:25:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data=[(1,"a",30),(2,"b",32)]
user_schema=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("age",IntegerType())
])
df=spark.createDataFrame(data,user_schema)

In [3]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a| 30|
|  2|   b| 32|
+---+----+---+



In [4]:
data2=[
    (1,"Alice",["Reading","Hiking"]),
    (2,"Bob",["Swimming","Gardening","Painting"]),
    (3,"Charlie",["Cooking"]),
    (4, "David", ["Photography", "Skiing", "Cooking"])
    
]

In [7]:
user_schema2=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("Hobby",ArrayType(StringType()))
])

In [8]:
df2=spark.createDataFrame(data2,user_schema2)

In [9]:
df2.show()

+---+-------+--------------------+
| id|   name|               Hobby|
+---+-------+--------------------+
|  1|  Alice|   [Reading, Hiking]|
|  2|    Bob|[Swimming, Garden...|
|  3|Charlie|           [Cooking]|
|  4|  David|[Photography, Ski...|
+---+-------+--------------------+



In [10]:
from pyspark.sql.functions import *

In [13]:
df2.select("id","name",explode("Hobby")).show()#used to send data in different row

+---+-------+-----------+
| id|   name|        col|
+---+-------+-----------+
|  1|  Alice|    Reading|
|  1|  Alice|     Hiking|
|  2|    Bob|   Swimming|
|  2|    Bob|  Gardening|
|  2|    Bob|   Painting|
|  3|Charlie|    Cooking|
|  4|  David|Photography|
|  4|  David|     Skiing|
|  4|  David|    Cooking|
+---+-------+-----------+



In [15]:
df2.withColumn("newHobby",explode("Hobby")).show()

+---+-------+--------------------+-----------+
| id|   name|               Hobby|   newHobby|
+---+-------+--------------------+-----------+
|  1|  Alice|   [Reading, Hiking]|    Reading|
|  1|  Alice|   [Reading, Hiking]|     Hiking|
|  2|    Bob|[Swimming, Garden...|   Swimming|
|  2|    Bob|[Swimming, Garden...|  Gardening|
|  2|    Bob|[Swimming, Garden...|   Painting|
|  3|Charlie|           [Cooking]|    Cooking|
|  4|  David|[Photography, Ski...|Photography|
|  4|  David|[Photography, Ski...|     Skiing|
|  4|  David|[Photography, Ski...|    Cooking|
+---+-------+--------------------+-----------+



In [20]:
df3=df2.withColumn("Hobby",explode("Hobby")).withColumn("ingestion_date",current_timestamp()).show(truncate=False)
    #hobby is same so data overwrite  and no new column is created and changes is made to original column

+---+-------+-----------+-----------------------+
|id |name   |Hobby      |ingestion_date         |
+---+-------+-----------+-----------------------+
|1  |Alice  |Reading    |2023-09-22 04:49:30.822|
|1  |Alice  |Hiking     |2023-09-22 04:49:30.822|
|2  |Bob    |Swimming   |2023-09-22 04:49:30.822|
|2  |Bob    |Gardening  |2023-09-22 04:49:30.822|
|2  |Bob    |Painting   |2023-09-22 04:49:30.822|
|3  |Charlie|Cooking    |2023-09-22 04:49:30.822|
|4  |David  |Photography|2023-09-22 04:49:30.822|
|4  |David  |Skiing     |2023-09-22 04:49:30.822|
|4  |David  |Cooking    |2023-09-22 04:49:30.822|
+---+-------+-----------+-----------------------+

